In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import load_img, img_to_array, array_to_img
from tensorflow.keras.callbacks import EarlyStopping
import os

In [2]:
def resize_images_in_directory(directory, target_size=(100, 100)):
    """
    Resize all images in the directory to the target size if necessary.
    If an image is already the target size, it will not be resized.

    Args:
        directory (str): Path to the directory containing images.
        target_size (tuple): Target size for the images (width, height).
    """
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(('png', 'jpg', 'jpeg')):
                file_path = os.path.join(root, file)
                try:
                    # Load image
                    img = load_img(file_path)
                    # Check if resizing is needed
                    if img.size != target_size:
                        # Resize image
                        img_resized = img.resize(target_size)
                        # Save resized image
                        img_resized.save(file_path)
                        print(f"Resized: {file_path}")
                    else:
                        print(f"Already 100x100: {file_path}")
                except Exception as e:
                    print(f"Error processing {file_path}: {e}")


In [3]:
# Set the path to the dataset folder
dataset_dir = 'X:/RAHMAN/DatasetBaruu'

# Resize semua gambar di folder dataset menjadi 100x100 jika diperlukan
resize_images_in_directory(dataset_dir, target_size=(100, 100))

Already 100x100: X:/RAHMAN/DatasetBaruu\Alpukat\45_100.jpg
Already 100x100: X:/RAHMAN/DatasetBaruu\Alpukat\46_100.jpg
Already 100x100: X:/RAHMAN/DatasetBaruu\Alpukat\47_100.jpg
Already 100x100: X:/RAHMAN/DatasetBaruu\Alpukat\48_100.jpg
Already 100x100: X:/RAHMAN/DatasetBaruu\Alpukat\49_100.jpg
Already 100x100: X:/RAHMAN/DatasetBaruu\Alpukat\4_100.jpg
Already 100x100: X:/RAHMAN/DatasetBaruu\Alpukat\50_100.jpg
Already 100x100: X:/RAHMAN/DatasetBaruu\Alpukat\51_100.jpg
Already 100x100: X:/RAHMAN/DatasetBaruu\Alpukat\52_100.jpg
Already 100x100: X:/RAHMAN/DatasetBaruu\Alpukat\53_100.jpg
Already 100x100: X:/RAHMAN/DatasetBaruu\Alpukat\54_100.jpg
Already 100x100: X:/RAHMAN/DatasetBaruu\Alpukat\55_100.jpg
Already 100x100: X:/RAHMAN/DatasetBaruu\Alpukat\56_100.jpg
Already 100x100: X:/RAHMAN/DatasetBaruu\Alpukat\57_100.jpg
Already 100x100: X:/RAHMAN/DatasetBaruu\Alpukat\58_100.jpg
Already 100x100: X:/RAHMAN/DatasetBaruu\Alpukat\59_100.jpg
Already 100x100: X:/RAHMAN/DatasetBaruu\Alpukat\5_100.jpg

In [4]:
# Image data generator untuk augmentasi gambar
train_datagen = ImageDataGenerator(
    rescale=1./255,            # Normalisasi gambar
    rotation_range=30,         # Rotasi acak gambar
    width_shift_range=0.2,     # Pergeseran horizontal
    height_shift_range=0.2,    # Pergeseran vertikal
    shear_range=0.2,           # Shearing
    zoom_range=0.2,            # Zooming
    horizontal_flip=True,      # Pembalikan horizontal
    fill_mode='nearest'       # Mengisi area kosong setelah transformasi
)

In [5]:
# Membaca data training dari folder dan mengorganisasi ke dalam label
train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(100, 100),    # Ukuran gambar yang akan diproses
    batch_size=32,             # Jumlah gambar per batch
    class_mode='categorical'   # Untuk klasifikasi multi-kelas
)

Found 1827 images belonging to 5 classes.


In [6]:
# Membangun model CNN
model = Sequential()

In [7]:
# Layer konvolusi pertama
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))


In [8]:
# Layer konvolusi kedua
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [9]:
# Layer konvolusi ketiga
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [10]:
# Flatten layer untuk mengubah data 2D menjadi 1D
model.add(Flatten())

In [11]:
# Fully connected layer pertama
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Dropout untuk mencegah overfitting


In [12]:
# Output layer dengan 5 kelas buah
model.add(Dense(5, activation='softmax'))

In [13]:
# Kompilasi model dengan optimizer Adam
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
# Membuat callback EarlyStopping
early_stop = EarlyStopping(
    monitor='loss',            # Metode untuk memantau (misalnya, 'val_loss' atau 'loss')
    patience=3,                # Jumlah epoch tanpa peningkatan sebelum berhenti
    restore_best_weights=True  # Mengembalikan bobot terbaik sebelum berhenti
)

In [15]:
# Melatih model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=100,                 # Maksimum jumlah epoch
    callbacks=[early_stop],    # Menyisipkan callback
    verbose=1
)

Epoch 1/100
57/57 [==============================] - 24s 405ms/step - loss: 1.4142 - accuracy: 0.3855
Epoch 2/100
57/57 [==============================] - 18s 316ms/step - loss: 1.1983 - accuracy: 0.4836
Epoch 3/100
57/57 [==============================] - 17s 303ms/step - loss: 1.1041 - accuracy: 0.5387
Epoch 4/100
57/57 [==============================] - 17s 302ms/step - loss: 0.9880 - accuracy: 0.5967
Epoch 5/100
57/57 [==============================] - 17s 306ms/step - loss: 0.9066 - accuracy: 0.6435
Epoch 6/100
57/57 [==============================] - 18s 308ms/step - loss: 0.8291 - accuracy: 0.6869
Epoch 7/100
57/57 [==============================] - 17s 305ms/step - loss: 0.7198 - accuracy: 0.7448
Epoch 8/100
57/57 [==============================] - 18s 318ms/step - loss: 0.6434 - accuracy: 0.7833
Epoch 9/100
57/57 [==============================] - 18s 313ms/step - loss: 0.6095 - accuracy: 0.7989
Epoch 10/100
57/57 [==============================] - 17s 303ms/step - loss: 0.547

In [16]:
# Menyimpan model
model.save('fruit_classifier_model.h5')

print("Model training selesai dan disimpan.")

Model training selesai dan disimpan.


x:\RAHMAN\.venv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [17]:
model_new=tf.keras.models.load_model('fruit_classifier_model.h5')
model_new.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 49, 49, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 23, 23, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 21, 21, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 10, 10, 128)       0

In [18]:
# Check its accuracy:
model_new = tf.keras.models.load_model('fruit_classifier_model.h5')
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

NameError: name 'new_model' is not defined